# Saddle Flow Dynamics

$\min\limits_x \frac{1}{2}\|Xx-b\|^2+\lambda\|x\|_1$

In [9]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import gurobipy as gp 
from gurobipy import GRB
import os

np.random.seed(2023)

In [10]:
np.random.seed(2023)
# generate positive semi-definite matrix
def gen_psd(n):
    X = np.random.rand(n, n)
    B = np.dot(X, X.transpose())
    B_pos = B - np.linalg.eigvalsh(B).min() * np.eye(n)
    X_pos = np.sqrt(B_pos)
    return X_pos

# X = gen_psd(n) 
# lf = np.linalg.eigvalsh(np.dot(X, X.transpose())).max()
# mf = np.linalg.eigvalsh(np.dot(X, X.transpose())).min()

# lambd =  4.0
# b = np.random.normal(-1,1,(n))
# ## initial point
# x0 = np.random.rand(dim)

# print(X,l,b,x0)
# np.save(f'Para/lambd.npy', lambd) 
# np.save(f'Para/matrix_X.npy', X) 
# np.save(f'Para/vector_b.npy', b)
# np.save(f'Para/vector_x0.npy', x0)
# param_dict = {
#     'lambd': lambd,
#     'b_vector': b,
#     'x0_vector': x0,
#     'matrix_X': X
# }
# np.savez(
#     f'Para/parameters.npz',
#     **param_dict,
#     allow_pickle=True
# )

In [11]:
today = datetime.date.today()
# parameters
n = 5
dim = 6*n
I = np.eye(n)
time = 800
step = 2000
time_eval = np.linspace(0,time,step)

try:
    os.makedirs('Para', exist_ok=True)
    
    X = np.load('Para/matrix_X.npy')
    b = np.load('Para/vector_b.npy')
    x0 = np.load('Para/vector_x0.npy')
    lambd = np.load('Para/lambd.npy')

    lf = np.linalg.eigvalsh(X @ X.T).max()
    mf = np.linalg.eigvalsh(X @ X.T).min()
    
except FileNotFoundError:
    print("Error:Not Find Parameters")

alpha_list =  [0.1/lf, 1.0/lf, 2.5/lf, 4.0/lf,6.0/lf]
rho_list = [1.0]


In [12]:
# f(x)
def f(x, X, b):
    return np.linalg.norm(X@x-b, 2)**2/2.0

def grad_f(x, X, b):
    return X.T@(X@x-b)


In [13]:
def v_star(u_bar, v_bar,lamb, alph, rho):
    x_bar= u_bar[:n]
    z_bar_pos= u_bar[n:2*n]
    z_bar_neg= u_bar[2*n:]
    y_bar = v_bar[:n]
    w_bar_pos = v_bar[n:2*n]
    w_bar_neg = v_bar[2*n:]

    ############### gurobi: v_star ################
    model = gp.Model("Proximal_Saddle")
    model.setParam('OutputFlag', 0) 
    y = model.addMVar(n, lb=-gp.GRB.INFINITY, name='y')
    w_pos= model.addMVar(n, name='w_pos')
    w_neg= model.addMVar(n, name='w_neg')   
    
    lambd = lamb
    alpha = alph


    fun1 = 0.5 * ((X@(x_bar-alpha*y)-b)@(X@(x_bar-alpha*y)-b)).sum()
    loss = lambd * (z_bar_pos+ z_bar_neg + alpha*(w_pos + w_neg)).sum()
    fun2 = (y@x_bar).sum()-((y+w_pos)@z_bar_pos).sum()+((y-w_neg)@z_bar_neg).sum()
    fun3 = alpha*(y@y + (y + w_pos)@(y + w_pos) + (y - w_neg)@(y - w_neg)).sum()
    fun4 = rho/2.0*((y-y_bar)@(y-y_bar) + (w_pos-w_bar_pos)@(w_pos-w_bar_pos)+ (w_neg-w_bar_neg)@(w_neg-w_bar_neg)).sum()
    # print(fun1, loss, fun2, fun3, fun4)
    
    # set objective
    model.setObjective(fun1 + loss + fun2 - fun3 - fun4, GRB.MAXIMIZE)
    # set constraints
    model.addConstr(w_pos >= 0)
    model.addConstr(w_neg >= 0)

    model.update()
    model.Params.NonConvex = 2
    model.optimize()

    y_optimal = np.zeros(n)
    w_pos_optimal = np.zeros(n)
    w_neg_optimal = np.zeros(n)
    if model.status == GRB.OPTIMAL:
        # Access the optimal variable values
        for i in range(n):
            y_optimal[i] = model.getVarByName('y[{}]'.format(i)).X
            w_pos_optimal[i] = model.getVarByName('w_pos[{}]'.format(i)).X
            w_neg_optimal[i] = model.getVarByName('w_neg[{}]'.format(i)).X
    else:
        print("Optimization did not converge to an optimal solution.")
    return y_optimal, w_pos_optimal, w_neg_optimal

In [14]:
## ODE
# saddle flow dynamics(not the projected version)
def dx(x, t, lambd, alpha, rho):
    x_bar, z_bar_pos, z_bar_neg, y_bar, w_bar_pos, w_bar_neg = x[:n], x[n:2*n], x[2*n:3*n], x[3*n:4*n], x[4*n:5*n], x[5*n:]

    u_bar = np.concatenate((x_bar, z_bar_pos, z_bar_neg))
    v_bar = np.concatenate((y_bar, w_bar_pos, w_bar_neg))
    y_star, w_star_pos, w_star_neg = v_star(u_bar, v_bar, lambd, alpha, rho)

    # print("x_bar: ", x_bar,'y_star: ', y_star,'dx_bar: ', -(grad_f(x_bar - alpha*y_star) + y_star))
    dx_bar = -(grad_f(x_bar - alpha*y_star, X, b) + y_star)
    dy_bar = rho*(y_star - y_bar)
    dz_bar_pos = -(lambd*np.ones(n) - y_star - w_star_pos)
    dz_bar_neg = -(lambd*np.ones(n) + y_star - w_star_neg)
    dw_bar_pos = rho*(w_star_pos - w_bar_pos)
    dw_bar_neg = rho*(w_star_neg - w_bar_neg)
    for i in range(n):
        if w_bar_pos[i] <= 0:
            if dw_bar_pos[i] < 0:
                dw_bar_pos[i] = 0
        if w_bar_neg[i] <= 0:
            if dw_bar_neg[i] < 0:
                dw_bar_neg[i] = 0
    dynamics = np.concatenate((dx_bar, dz_bar_pos, dz_bar_neg, dy_bar, dw_bar_pos, dw_bar_neg)).reshape(6*n,).tolist()
    return dynamics

In [15]:
# compute the optimal solution to the equivalent form of Lasso regression problem
def gurobi_lasso(X, b, lambd):
    dim = 3*n
    model = gp.Model("Lasso")
    model.setParam('OutputFlag', 0) 
    u = model.addMVar(dim, lb=-gp.GRB.INFINITY, name='u')


    x = u[:n]
    z = u[n:]
    obj = 0.5 * ((X@x-b)@(X@x-b)).sum() + lambd*np.ones(2*n)@z
    # set objective
    model.setObjective(obj, GRB.MINIMIZE)
    # set constraints
    I = np.eye(n)
    # B = np.block([-I, I])
    model.addConstr(x - u[n:2*n] + u[2*n:3*n] == 0)
    model.addConstr(u[n:] >= 0)

    model.update()
    model.optimize()

    u_opt = np.zeros(dim)
    v_opt = np.zeros(dim)
    if model.status == GRB.OPTIMAL:
        # Access the optimal variable values
        for i in range(dim):
            u_opt[i] = model.getVarByName('u[{}]'.format(i)).X
    else:
        print("Optimization did not converge to an optimal solution.")
    flag = 0
    for con in model.getConstrs():
        v_opt[flag] = con.Pi
        flag += 1
    v_opt[:n] = -v_opt[:n]
    return u_opt, v_opt

In [16]:
res = {}
u_opt, v_opt = gurobi_lasso(X,b,lambd)
print("Primal_opt: ", u_opt,"\n","Dual_opt:", v_opt)
for alpha in alpha_list:
    for rho in rho_list:
        print("alpha: ", alpha)
        print("rho: ", rho)
        # Initialize inner dictionary for alpha if it doesn't exist
        if alpha not in res:
            res[alpha] = {}

        # Compute and store the result in the nested dictionary
        res[alpha][rho] = odeint(dx, x0, time_eval, args=(lambd, alpha, rho))


Primal_opt:  [-2.83697848e-08 -5.89781782e-01 -3.19751533e-09 -2.43868534e-10
 -7.96403353e-11  2.79790601e-12  2.79717345e-12  2.89320527e-12
  3.10171256e-12  3.29209308e-12  2.83725832e-08  5.89781782e-01
  3.20040895e-09  2.46971604e-10  8.29324290e-11] 
 Dual_opt: [-3.95788639 -4.         -3.63690454 -2.865819   -3.44762093  7.95788639
  8.          7.63690454  6.865819    7.44762093  0.04211361  0.
  0.36309546  1.134181    0.55237907]
alpha:  0.006063757177382553
rho:  1.0
alpha:  0.060637571773825526
rho:  1.0
alpha:  0.15159392943456382
rho:  1.0
alpha:  0.2425502870953021
rho:  1.0
alpha:  0.36382543064295314
rho:  1.0


In [18]:
import pickle
now = datetime.datetime.now()
# Pickling the object
Storage = {'result':res,'rho_list': rho_list, 'alpha_list': alpha_list, 'time_step': time_eval, 'n': n, 'u_opt': u_opt, 'v_opt': v_opt, 'l':lf}
with open('Result/para_results_{}_parameters.pkl'.format(now), 'wb') as file:
    pickle.dump(Storage, file)